In [5]:
import os 

In [6]:
%pwd

'c:\\Users\\admin\\Desktop\\Predictive Maintenance\\Predictive_Maintenance_With_MLops\\research'

In [7]:
os.chdir("../")

In [8]:
%pwd

'c:\\Users\\admin\\Desktop\\Predictive Maintenance\\Predictive_Maintenance_With_MLops'

In [9]:
# writing entity : is just like a return type 


from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [1]:
from src.Mlflow_Project.constants import *
from src.Mlflow_Project.utils.utility import FileOperations

ModuleNotFoundError: No module named 'src'

In [11]:
# configuration manager class , will read all yaml files 

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = FileOperations.read_yaml(config_filepath)
        self.params = FileOperations.read_yaml(params_filepath)
        self.schema = FileOperations.read_yaml(schema_filepath)

        FileOperations.create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        FileOperations.create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [12]:
# Updating our components 

import os 
import urllib.request as request
import zipfile 
from src.Mlflow_Project import logger 
from src.Mlflow_Project.utils.utility import FileOperations

In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {FileOperations.get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [14]:
# update the pipeline 

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-10-02 11:04:32,496: INFO: yaml file: config\config.yaml loaded successfully]
[2023-10-02 11:04:32,504: INFO: yaml file: params.yaml loaded successfully]
[2023-10-02 11:04:32,504: INFO: yaml file: schema.yaml loaded successfully]
[2023-10-02 11:04:32,504: INFO: created directory at: artifacts]
[2023-10-02 11:04:32,512: INFO: created directory at: artifacts/data_ingestion]
[2023-10-02 11:04:40,205: INFO: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 12944209
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "86e36c0bd5538d05fd7d8f8e4e193c0ef25e0594642f915185d23568c543dcc8"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: B590:28EF72:108E2:1295A:651A5668
Accept-Ranges: bytes
Date: Mon, 02 Oct 2023 05:34:34 GMT
Via: 1.1 varnish
X-Served-By